In [24]:
import gseapy as gp
import pandas as pd

import h5py
import numpy as np
import pandas as pd
from collections import defaultdict
import phate
import scanpy as sc

from anndata import AnnData

import plotly.express as px
import plotly.graph_objects as go

# Build GSEA table for all tumor-cluster pairs

In [25]:
threshold = None

In [26]:
gsea_table = pd.DataFrame()

In [ ]:
for tumor in [1,2,3,4]:
    de_genes_tumor_df = pd.read_csv("data/interim/MK_genes_TUMOR{}.csv".format(tumor))
    de_genes_by_cluster = de_genes_tumor_df.groupby("cluster")["gene"].apply(lambda x: "|".join(x.unique()))
    
    for cluster in de_genes_by_cluster.index:
        gene_list = de_genes_by_cluster[cluster].split("|")
        tumor_cluster = str(tumor) + "_" + str(cluster)

        enr = gp.enrichr(gene_list=gene_list,
                     gene_sets=['GO_Biological_Process_2018',
                                'GO_Cellular_Component_2018',
                                'GO_Molecular_Function_2018'],
                     no_plot=True,
                     cutoff=0.05 # test dataset, use lower value from range(0,1)
                    )
        if threshold:
            enr.results = enr.results[enr.results["Adjusted P-value"] < threshold]
        enr_results = enr.results.set_index("Term")
        
        for geneset in enr_results.index:
            gsea_table.loc[geneset, tumor_cluster] = enr_results.loc[geneset, "Adjusted P-value"]

In [ ]:
fig = go.Figure(data=go.Heatmap(
                z=gsea_table,
                x=gsea_table.columns,
                y=gsea_table.index,
                hoverongaps = False,))
fig.update_layout(
    title="Gene set enrichment of DE genes by tumor_cluster",
    autosize=True,
    width=1000,
    height=800,
)
fig.show()

# Filter Clusters by biomarker expression of cells

In [ ]:
DATA_F = '../data/GSE103224.h5'
BIOMARKER_F = '../data/glioma_survival_associated_genes_Fatai.csv'

with h5py.File(DATA_F, 'r') as f:
    CELLS = [
        str(x)[2:-1]
        for x in f['cell'][:]
    ]
    TUMORS = [
        str(x)[2:-1]
        for x in f['tumor'][:]
    ]
    GENE_IDS = [
        str(x)[2:-1]
        for x in f['gene_id'][:]
    ]
    GENE_NAMES = [
        str(x)[2:-1]
        for x in f['gene_name'][:]
    ]
    
# Map each cell to its index in the data matrix
CELL_TO_INDEX = {
    cell: index
    for index, cell in enumerate(CELLS)
}

# Map each tumor to its indices in the data matrix
TUMOR_TO_INDICES = defaultdict(lambda: [])
for index, tumor in enumerate(TUMORS):
    TUMOR_TO_INDICES[tumor].append(index)
TUMOR_TO_INDICES = dict(TUMOR_TO_INDICES)

def counts_matrix_for_tumor(tumor):
    indices = TUMOR_TO_INDICES[tumor]
    with h5py.File(DATA_F, 'r') as f:
        counts = f['count'][indices]
    cells = list(np.array(CELLS)[indices])
    return counts, cells

In [ ]:
tumor_dfs = {}
TUMORS = np.unique(TUMORS)
for tumor in TUMORS:
    print(tumor)
    counts, cells = counts_matrix_for_tumor(tumor)
    ad = AnnData(
            X=counts, 
            obs=pd.DataFrame(data=cells, columns=['cell']),
            var=pd.DataFrame(
                index=GENE_NAMES, 
                data=GENE_NAMES, 
                columns=['gene_name']
            )
        )
    sc.pp.normalize_total(ad, target_sum=1e6)
    sc.pp.log1p(ad)
    tumor_dfs[tumor] = ad

In [32]:
enr.results

,Adjusted P-value,Combined Score,Gene_set,Genes,Odds Ratio,Old Adjusted P-value,Old P-value,Overlap,P-value
Term,,,,,,,,,
regulation of cell proliferation (GO:0042127),0.000021,119.406906,GO_Biological_Process_2018,PDGFRA;CDKN2A;IGFBP3;LIFR;IRS2;CDC6;EGFR;VEGFA...,6.177606,0,0,16/740,4.032042e-09
negative regulation of apoptotic process (GO:0043066),0.000029,139.994319,GO_Biological_Process_2018,PDGFRA;FZD3;MGMT;DKK1;EGFR;VEGFA;SFRP1;PIK3CA;...,7.658321,0,0,13/485,1.151025e-08
positive regulation of cellular process (GO:0048522),0.000044,125.098116,GO_Biological_Process_2018,PDGFRA;LIFR;IRS2;CDC6;DKK1;EGFR;VEGFA;SFRP1;AC...,7.156620,0,0,13/519,2.561581e-08
G1 DNA damage checkpoint (GO:0044783),0.000036,1985.799593,GO_Biological_Process_2018,CCND1;CDK2;WAC;TP53,114.285714,0,0,4/10,2.843212e-08
G1/S transition of mitotic cell cycle (GO:0000082),0.000084,310.781696,GO_Biological_Process_2018,CCNA1;CCND1;CDKN2A;CCNE1;POLE3;CDK2;CDC6,19.047619,0,0,7/105,8.204163e-08
...,...,...,...,...,...,...,...,...,...
"transcriptional activator activity, RNA polymerase II transcription regulatory region sequence-specific binding (GO:0001228)",1.000000,0.459770,GO_Molecular_Function_2018,TP53,1.006036,0,0,1/284,6.331734e-01
transcription regulatory region sequence-specific DNA binding (GO:0000976),1.000000,0.431396,GO_Molecular_Function_2018,TP53,0.978474,0,0,1/292,6.434657e-01
sequence-specific DNA binding (GO:0043565),1.000000,0.206478,GO_Molecular_Function_2018,PPARGC1A,0.725163,0,0,1/394,7.522146e-01
